## Reproduce datasets for Scheme B

Paper: "Statistical supervised meta-ensemble algorithm for data linkage"

Kha Vo, Jitendra Jonnagaddala, Siaw-Teng Liaw

February 2019

Jounal of Biomedical Informatics

In [17]:
inputfile = 'ePBRN_F_original' # change between 'ePBRN_D_original' or 'ePBRN_F_original'
outputfile = 'ePBRN_F_dup' # change between 'ePBRN_D_dup' or 'ePBRN_F_dup'

In [18]:
import pandas as pd
import numpy as np
import random
from numpy.random import choice
import sys

In [19]:
# (andrew) Remove pd warning that is taking up tons of output logs
# https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [20]:
# (andrew) Need to add above directory to path since these notebooks run in their relative
# directory
import sys
sys.path.append("..")
from utils import set_random_seed
set_random_seed()

In [21]:
# Input the percentage of  [2, 3, 4] shared records in one linkage:
count_shared = [1.68+21.0659, 1.9986 + 0.0471, 0.05]

# Assigning the weights for each type of error:

abr = 1 # abbreviation on surname: Michael -> M
jwd1 = 1 # join with dash: John Peter -> John-Peter, join surname and given name into surname
jwd2 = 1 # join with dash: John Peter -> John-Peter, join surname and given name into given name
jwb1 = 1 # join with blank: 
jwb2 = 1 # join with blank: 
drf = 1 # drop all tokens in any field
dlc1 = 1 # drop last character in surname: Peter -> Pete
dlc2 = 1 # drop last character in given name
swn = 1 # swap surname and given name: John Peter -> Peter John
swd = 1 # swap day and month fields: 12/04 -> 04/12
rsd = 1 # reset day and month: 12/04/1991 -> 01/01/1991
chy = 1 # change year of birth by a margin of (+/-)5 
chz = 1 # change any number of digit from zip code
mar = 1 # change the whole token of surname: Mary Ward -> Mary Winston
twi = 1 # duplicate all fields except given name: Micheal Williams -> Leo Williams
add = 1 # change the whole 3 fields of address by randomly replacing each field by any other row

all_error_types = ['abr','jwd1','jwd2','jwb1','jwb2' ,'drf','dlc1','dlc2','swn',
                   'rsd','chy','chz','mar','twi','add']
all_error_weights = [abr, jwd1, jwd2, jwb1, jwb2, drf, dlc1, dlc2, swn, rsd, chy, chz, mar, twi, add]
all_error_weights = all_error_weights/sum(np.asarray(all_error_weights))

# COMMENT(alecmori): If all errors {jwd1, jwd2, jwb1, jwb2} are enabled,
# rc['surname'] == rc[surname] + "-" + rc[given_name] + "-  " (it keeps augmenting the last name).
# This has major negative repercussions in later downstream branches.
# We should clean this up for experiments.
def process_record(rc):
    no_error = np.random.poisson(1, 1)
    errortypes = choice(all_error_types, no_error, p=all_error_weights)
    for errortype in errortypes:
        if errortype == 'abr':
            if len(rc["surname"])>0:
                #rc["surname"] = rc["surname"][0]
                rc.at["surname"] = rc["surname"][0]
        if errortype == 'jwd1':
            rc.at["surname"] = rc["surname"] + '-' + rc["given_name"]
            rc.at["given_name"] = ''
        if errortype == 'jwd2':
            rc.at["given_name"] = rc["surname"] + '-' + rc["given_name"]
            rc.at["surname"] = ''
        if errortype == 'jwb1':
            rc.at["surname"] = rc["surname"] + ' ' + rc["given_name"]
            rc.at["given_name"] = ''
        if errortype == 'jwb2':
            rc.at["given_name"] = rc["surname"] + ' ' + rc["given_name"]
            rc.at["surname"] = ''
        if errortype == 'drf':    
            selected_field = random.choice(all_fields)
            rc.at[selected_field] = ''
        if errortype == 'dlc1':
            if len(rc["surname"])>0:
                rc.at["surname"] = rc['surname'][0:-1]
        if errortype == 'dlc2':
            if len(rc["given_name"])>0:
                rc.at["given_name"] = rc['given_name'][0:-1]
        if errortype == 'swn':
            temp = rc['given_name']
            rc.at["given_name"] = rc['surname']
            rc.at["surname"] = temp
        if errortype == 'swd': 
            temp = rc['day']
            rc.at['day'] = rc['month']
            rc.at['month'] = temp
        if errortype == 'rsd':
            rc.at['day'] = '01'
            rc.at['month'] = '01'
        if errortype == 'chy': 
            if rc.at['year'] != 'NaT' and rc['year'] != '':
                margin = random.choice(range(-5,5))
                rc.at['year'] = str( int(rc['year']) + margin)
        if errortype == 'chz':
            if len(str(rc['postcode']))== 4:
                selected_digit = random.choice(range(4))
                code = list(str(rc['postcode']))
                code[selected_digit] = str( random.choice(range(9)))
                rc.at['postcode'] = int(''.join(code))
        if errortype == 'mar':
            rc.at["surname"] = df.iloc[random.choice(range(len(df)))]['surname']
        if errortype == 'twi':
            rc.at["given_name"] = df.iloc[random.choice(range(len(df)))]['given_name']
        if errortype == 'add':
            rc.at['address_1'] = df.iloc[random.choice(range(len(df)))]['address_1']
            rc.at['address_2'] = df.iloc[random.choice(range(len(df)))]['address_2']
            rc.at['street_number'] = random.choice(range(500))
    return rc

In [22]:
# Preprocess
df = pd.read_csv(
    f"ePBRN_Datasets/{inputfile}.csv", 
    parse_dates=["date_of_birth"])

df["rec_id"] = range(len(df))
df['rec_id'] = df['rec_id'].astype(str)
df['day'] = df['date_of_birth'].dt.strftime('%d')
df['month'] = df['date_of_birth'].dt.strftime('%m')
# Comment(alecmori): Added fillna here due to a missing data point.
df['year'] = df['date_of_birth'].dt.strftime('%Y').fillna(0)
df['postcode'] =   df['postcode'].fillna('0000')
df['postcode'] = df['postcode'].astype(int)
df['street_number'] =   df['street_number'].fillna('0')
df['street_number'] = df['street_number'].astype(int)
df = df.drop(["age", "phone_number", "soc_sec_id", "blocking_number", "date_of_birth"], axis=1)
for col in ["surname", "given_name", "address_1", "address_2", "day", "month"]:
    df[col] = df[col].fillna('')
    df[col] = df[col].astype(str)
df["match_id"] = range(len(df))
all_fields = df.columns.values.tolist()
all_fields.remove('rec_id')

print("Original dataset length:",len(df))
df.head()


Original dataset length: 11100


,rec_id,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,day,month,year,match_id
0,0,jenna,kilpin,179,mcfarlan place,,hillarys,2768,vic,26,02,1950,0
1,1,bianca,randazzo,37,lindrum crescent,sunshine,forster,2281,wa,07,04,1988,1
2,2,james,borlase,75,,rocklea,casula,2460,qld,03,09,1913,2
3,3,nicholas,beeton,20,mugga way,,hawthorn,2480,vic,22,06,1999,3
4,4,megan,footner,4,jewell close,,taylors lakes,3129,tas,22,09,1912,4


In [23]:
# Generate random indices for linkages
leng = len(df)
print("Process", inputfile, ", total records:", leng, "...")
# Comment(alecmori): Why are we doing this hacky thing to count how many of each link there are
# instead of actually counting them.
no_double_linked = int(leng*count_shared[0]/100)
no_triple_linked = int(leng*count_shared[1]/100)
no_quad_linked = int(leng*count_shared[2]/100)
list_double_linked = random.sample(range(leng),k=no_double_linked)
remain = [item for item in range(leng) if item not in list_double_linked]
list_triple_linked = random.sample(remain,k=no_triple_linked)
remain = [item for item in remain if item not in list_triple_linked]
list_quad_linked = random.sample(remain,k=no_quad_linked)
print("Double links:", no_double_linked,". Triple links:",no_triple_linked,". Quad links:",no_quad_linked)
print("Total records after generated:", leng + no_double_linked + no_triple_linked*2 + no_quad_linked*3)
print("Matched pairs:", no_double_linked + no_triple_linked*3 + no_quad_linked*6)

Process ePBRN_F_original , total records: 11100 ...
Double links: 2524 . Triple links: 227 . Quad links: 5
Total records after generated: 14093
Matched pairs: 3235


In [24]:
# Main steps
df1 = df
j = 0
for list_linked in [list_double_linked, list_triple_linked, list_quad_linked]:
    j = j + 1
    for i in list_linked:
        for k in range(j):
            record_to_process = df.iloc[i]
            processed_rc = process_record(record_to_process)
            rec_id = processed_rc["rec_id"]
            processed_rc.at["rec_id"] = f"{rec_id}-dup-{k}"
            df1 = df1.append(processed_rc)

# Save to disk    
outputpath = f"ePBRN_Datasets/{outputfile}.csv"
df1.to_csv(outputpath, index=False)
print("Saved to", outputpath)

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [25]:
df1.tail(25)

,rec_id,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,day,month,year,match_id
10389,10389-dup-0,taylor rachel,,11,carandini street,,wamberal,2027,act,24,09,1989,10389
10389,10389-dup-1,rachel,taylor,11,carandini street,,wamberal,2027,act,24,09,1989,10389
10065,10065-dup-0,leach-corey,,9,batchelor street,,enmore,3061,nsw,28,11,1903,10065
10065,10065-dup-1,corey,leach,9,batchelor street,,enmore,3061,nsw,28,11,1905,10065
995,995-dup-0,alysha,humeniuk,110,halfrey circuit,,fairfield,3094,nsw,14,02,1909,995
995,995-dup-1,alysha,humeniuk,110,halfrey circuit,,fairfield,3094,nsw,14,02,1909,995
7620,7620-dup-0,jamie,fagan,14,daintree crescent,,noonamah,4113,vic,01,06,1974,7620
7620,7620-dup-1,fagan,jamie,14,daintree crescent,,noonamah,4113,vic,01,06,1974,7620
607,607-dup-0,,porter ruby,5,kenyon circuit,ribbermont farm,grafton,6069,nsw,,,0,607
607,607-dup-1,ruby,porter,5,kenyon circuit,ribbermont farm,grafton,6069,nsw,01,01,0,607


In [26]:
len(df)

11100